# CONSEGNA GRUPPO 19
*   Demetrio Trimarco
*   Emilio Sorrentino
*   Francesco Rosa
*   Francesco Sabbarese

# SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## IMPORT


In [2]:
%load_ext tensorboard
from tensorflow import keras
%tensorflow_version 2.x
import tensorflow as tf
from keras.callbacks import TensorBoard
from datetime import datetime
from packaging import version
import os
import io
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import cv2
import json

In [ ]:
# import vggface models
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_vggface
!pip install keras_applications
from keras_vggface.vggface import VGGFace
import keras_vggface.utils

## VARIABLES

In [4]:
# MODEL_PATH = "/content/gdrive/MyDrive/CONSEGNA/TEST/----------"
MODEL_PATH = "/content/gdrive/MyDrive/CONSEGNA/TEST/model_group_19.h5"

TEST_SIZE = 169396
eval_tfrecord_file_name = "/content/gdrive/MyDrive/CONSEGNA/DATASETS/test_set.tfrecord"

BATCH_SIZE = 64

# RUN EVALUATION

## Parse tfrecord function

In [5]:
def decode_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [240, 240])
  image = tf.cast(image, tf.uint8)
  return image

def conv_normalize(image):
  return keras_vggface.utils.preprocess_input(image, 'channels_last', version=2)

def conv_BGR2RGB(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def conv_RGB2BGR(image):
  return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# parse tfrecord samples
def read_tfrecord(example):
    features = (
        {
          "image": tf.io.FixedLenFeature([], dtype=tf.string),
          "filename": tf.io.FixedLenFeature([], dtype=tf.string)
        }
    )

    example = tf.io.parse_single_example(example, features)

    image = example["image"]
    
    image = decode_image(image) # the decoded image has BGR channels
    
    image = tf.numpy_function(conv_BGR2RGB, [image], tf.uint8)
    image = tf.cast(image, dtype=tf.float32)
    # conv_normalize needs RGB images as input and returns BGR images as output
    image = tf.numpy_function(conv_normalize, [image], tf.float32)

    filename = example["filename"]
    
    return image, filename

## Loading dataset function

In [6]:
# Load a dataset and parse the samples
def load_dataset(tfrecord_filename):
    dataset = tf.data.TFRecordDataset(tfrecord_filename)
    dataset = dataset.map(read_tfrecord)
    return dataset

## Pipeline creation function

In [7]:
def apply_pipeline(path, labeled = True):
    dataset = load_dataset(path)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

## Load model

In [8]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(MODEL_PATH, compile = False)
model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
          loss = tf.keras.losses.MeanAbsoluteError()
)
# Show the model architecture
# model.summary()

## Predict

In [ ]:
eval_dataset = apply_pipeline(eval_tfrecord_file_name)

y_pred = model.predict(eval_dataset,
    verbose = 1,
    batch_size = BATCH_SIZE,
    steps = TEST_SIZE/BATCH_SIZE,
    workers = 4,
    use_multiprocessing = True
)

In [10]:
y_pred_scalar = []
for y_hat in y_pred:
  y_pred_scalar.append(round(y_hat[0]))
y_pred_scalar = np.array(y_pred_scalar).T

In [ ]:
print(y_pred_scalar)

In [ ]:
from csv import reader

with open('/content/gdrive/MyDrive/CONSEGNA/TEST/test_filenames.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    i = 0
    for row in csv_reader:
      if y_pred_scalar[i] <= 18:
        print(row[0] + " - " + str(y_pred_scalar[i]))
      i = i + 1


## Result CSV creation

### Creation filename csv

In [ ]:
import csv

dataset = load_dataset(eval_tfrecord_file_name)
with open('test_filenames.csv', mode='w') as csvfile:
  out_csv = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  cnt = 0
  for _, filename in dataset:
    print(cnt)
    cnt = cnt + 1
    filename = str(filename.numpy()).split("'")[1]
    out_csv.writerow([filename])

### Creation result csv

In [ ]:
import csv

filenames = []
with open('/content/gdrive/MyDrive/CONSEGNA/TEST/test_filenames.csv', 'r') as csvinput:
  reader = csv.reader(csvinput)
  for row in reader:
    filenames.append(row)
print("filenames acquired")

with open('/content/gdrive/MyDrive/CONSEGNA/TEST/GROUP_19.csv', 'w') as csvoutput:
  out_csv = csv.writer(csvoutput, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  i = 0
  for filename in filenames:
    print(i)
    out_csv.writerow([filename[0], y_pred_scalar[i]])
    i = i + 1
print("csv file results created")